# DS-6 Data ETL  Pipeline Using Python (Extracting, Transforming and Loading)

* developing DataETL pipelines is one of the most valuable skills for DataEngineers.

* Data ETL is process where data is extracted from a place, then the data is transformed in some way, and then data is loaded into a database.So ETL (Extracting, Transforming and Loading the data)

* To Develop a Data ETL Pipeline  using python the first step is to collect data from data source. Let's use the FASHION -MINST
dataset provided by the Keras library to keep things Beginner Friendly.

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Collecting tensorflow
  Using cached tensorflow-2.12.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp39-cp39-win_amd64.whl (272.8 MB)
  Using cached grpcio-1.54.0-cp39-cp39-win_amd64.whl (4.1 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.12.2-py3-none-any.whl (5.6 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached jax-0.4.8-py3-none-any.whl
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached ml_dtypes-0.1.0-cp39-cp39-win_amd64.whl (120 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached google_auth-2.17.3-py2.py3-none-any.whl (178 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)


In [3]:

import tensorflow.keras as keras
(xtrain, ytrain), (xtest, ytest) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 2s 0us/step


* Befor moving forward Let's have a look at the shape of the data:

In [4]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


* Now Let's Clean and Transform the data. Here we will normalize the pixel values to be between 0 to 1 and reshape the data into a 4D tensor:

In [5]:
import numpy as np

xtrain = xtrain.astype('float32') / 255
xtest = xtest.astype('float32') / 255


xtrain= np.reshape(xtrain, (xtrain.shape[0],28,28,1))
xtest = np.reshape(xtest, (xtest.shape[0],28,28,1)) 

print(xtrain.shape)
print(xtrain.shape)
print(xtest.shape)
print(ytest.shape)

(60000, 28, 28, 1)
(60000, 28, 28, 1)
(10000, 28, 28, 1)
(10000,)


* Now Let's Load the data into a database, We ca nuse SQLite to create a database and load the data into it

In [11]:
import sqlite3

conn = sqlite3.connect('fashion_mnist.db')

conn.execute('''CREATE TABLE IF NOT EXISTS images
            (id INTEGER PRIMARY KEY  AUTOINCREMENT ,
            image BLOB NOT NULL,
            label INTERGER NOT NULL);''')

for i in range(xtrain.shape[0]):
    conn.execute('INSERT INTO images(image, label) VALUES (?,?)',
                [sqlite3.Binary(xtrain[i]), ytrain[i]])
                 
conn.commit()
for i in range(xtest.shape[0]):
                 conn.execute('INSERT INTO images(image, label) VALUES (?, ?)',
                             [sqlite3.Binary(xtest[i]), ytest[i]])
        
conn.commit()
conn.close()

In Above Code: 
    
    1. The First line imports a library called sqlite3, which allows us to work with SQLite databases in Python;
    2. We then create a connection to the database;
    3. next, we create a table in the databse called "imagees";
    4. We use a loop to llop thorugh each image in the training data and insert it into the "images" table (along with the labels);
    5. We use the commit() method to save the  the changes we made to the database;
    6. We then use another loop to loop through each image in the test data and insert it into the "images"table(along with Lables)
    7. We use the commit() method again to save the changes we made to the database;
    8. Finally we close the connection to the database;
    
    
    
* So this is how we can create a DataETL pipeline using python. Our ETL pipeline takes the Fashion MNIST dataset and stores it in an SQLite databse so that we can easily access and manipulate the data later.


### Now This is how you can read the data you stored on the SQLite database:

In [15]:
import  sqlite3
conn = sqlite3.connect("fashion_mnist.db")
cursor = conn.cursor()

cursor.execute('SELECT * FROM images')
rows = cursor.fetchall()

import pandas as pd
data = pd.read_sql_query('SELECT * FROM images', conn)
